In [1]:
import pandas as pd
import praw
import datetime        
from time import sleep
import random
import time



In [2]:
reddit = praw.Reddit(
    client_id='fKSnwxwrgn7DXbSDq8oHnw',         # Your app's client ID
    client_secret='xtz8CGEQjdB4SNbM9eHTp82eP0sIdQ', # Your app's client secret
    user_agent='Copenhagen University Data Collection Course, analyzing comment civility, Oliver',       # A descriptive user agent
    username='OkMinute',    # Your Reddit username
    password='Fodbold7'     # Your Reddit password
)

Version 7.7.1 of praw is outdated. Version 7.8.1 was released Friday October 25, 2024.


In [5]:
# Select the subreddit and fetch 5 top posts
subreddit = reddit.subreddit('carnivorediet')  # Replace 'europe' with your target subreddit
posts = list(subreddit.top(limit=15))  # Fetches the top 5 posts

# Collect basic information for each post
post_data = []
for post in posts:
    post_data.append({
        'Post ID': post.id,
        'Title': post.title,
        'Author': str(post.author),
        'Score': post.score,
        'Upvote Ratio': getattr(post, 'upvote_ratio', None),
        'Number of Comments': post.num_comments,
        'Created': pd.to_datetime(post.created_utc, unit='s')
    })


post_df = pd.DataFrame(post_data)

post_df

,Post ID,Title,Author,Score,Upvote Ratio,Number of Comments,Created
0,1cndy74,They were wrong…..,Done-with-work,1177,0.96,233,2024-05-08 20:03:50
1,1hv5y1e,9 months Carnivore Results,Cherry_DeVille,1064,0.99,190,2025-01-06 18:28:37
2,1fx64u9,Husband and I - Day 172 Carnivore,Cherry_DeVille,904,0.99,84,2024-10-06 02:04:39
3,1f1zkr8,Strict 117 Day Weightloss,Sea-Yogurtcloset-669,878,0.99,91,2024-08-26 21:19:02
4,1i0jjez,5 months in,uhhjes,879,0.99,147,2025-01-13 17:37:26
5,14mqffq,Carnivore saved my life and is healing TSW and...,Reasonable_Focus2468,874,0.99,89,2023-06-30 04:38:47
6,1hadifo,3 year carnivore,None,796,0.99,130,2024-12-09 16:23:20
7,1hwxkvl,I lost 65 pounds and could go on about how I f...,Not4me52,782,0.99,171,2025-01-08 22:57:02
8,1c10jfi,Wanted to show what just 3 months has done for...,The_meemster123,765,0.98,308,2024-04-10 23:52:49
9,1g263e2,2 months down 40 pounds.,Ryanmurf28,755,0.99,186,2024-10-12 18:09:44


In [6]:
# Define a function to collect comments from a specific post
def collect_comments(submission_id):
    # Fetch the submission object
    submission = reddit.submission(id=submission_id)

    # Replace all MoreComments objects to get full comment tree
    submission.comments.replace_more(limit=None)

    # Initialize list to store comment data
    comments_data = []

    # Use breadth-first traversal to collect comments with a single loop
    for comment in submission.comments.list():
        # Check if the comment has been deleted or removed
        if comment.body in ["[removed]", "[deleted]"]:
            continue

        # Add comment data to the list
        comments_data.append({
            'Post ID': submission.id,
            'Comment ID': comment.id,
            'Parent ID': comment.parent_id,
            'Author': str(comment.author),
            'Body': comment.body,
            'Score': comment.score,
            'Created': pd.to_datetime(comment.created_utc, unit='s'),
            'Depth': comment.depth  # Depth of the comment in the tree
        })

    # Convert collected data to a DataFrame
    comments_df = pd.DataFrame(comments_data)
    return comments_df






In [7]:
# Specify the ID of the post to scrape
 # Replace with the actual post ID
comments_df = collect_comments("1hcqd15")

In [8]:
comments_df

,Post ID,Comment ID,Parent ID,Author,Body,Score,Created,Depth
0,1hcqd15,m1q36dg,t3_1hcqd15,ew6281,You look fantastic. Good work.,20,2024-12-12 17:47:27,0
1,1hcqd15,m1qilad,t3_1hcqd15,deef1ve,"Besides the huge body fat loss, your face chan...",13,2024-12-12 19:06:07,0
2,1hcqd15,m1q20z4,t3_1hcqd15,Alibellygreenguts,You’re doing great 👍 \n Keep it up,17,2024-12-12 17:41:36,0
3,1hcqd15,m1qa8z5,t3_1hcqd15,Ftmzhra16,Any workout? You look fantastic.,8,2024-12-12 18:23:34,0
4,1hcqd15,m1qn4pf,t3_1hcqd15,_millenia_,"Niiiice! What were your meals each day, mainly...",7,2024-12-12 19:29:32,0
...,...,...,...,...,...,...,...,...
72,1hcqd15,m1r23lu,t1_m1qa8zh,Maleficent-Sleep9900,Yay! 🎉,1,2024-12-12 20:47:34,2
73,1hcqd15,m1rmj3a,t1_m1qa8zh,Gladstone233,"That’s huge, congratulations! Amazing results",1,2024-12-12 22:35:19,2
74,1hcqd15,m1w6v4i,t1_m1qa8zh,Bunbosa,May I ask what sort of mental health issues yo...,1,2024-12-13 18:08:37,2
75,1hcqd15,m1xnphe,t1_m1wltqx,OppositeDependent,Thank you for answering! Can’t wait to see ne...,1,2024-12-13 22:58:09,2


In [9]:
print(reddit.auth.limits)

{'remaining': 998.0, 'reset_timestamp': 1739437799.9652338, 'used': 2}
